# Trap and Spray Data

---

# Work book Overview

### Spray and Test Data analysis

+ Looks to see the impact spraying has on the mosquito population
+ Plots weekly maps to visualise how the population is growing
+ Shows where and at which point the spraying occurs

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt 
from bokeh.plotting import figure, output_file, save, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, Vendors
from bokeh.io import export_png
import math
from ast import literal_eval
from datetime import datetime
import time

sns.set_style("whitegrid")
%matplotlib inline

---

In [2]:
# Read in data
spray = pd.read_csv('./assets/spray_clean.csv')
train = pd.read_csv('./assets/train_clean.csv')

In [3]:
train.head(20)

,date,species,trap,latitude,longitude,nummosquitos,wnvpresent,year,month,day,day_of_week
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0,2007,May,29,Tue
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,1,0,2007,May,29,Tue
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,1,0,2007,May,29,Tue
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0,2007,May,29,Tue
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,4,0,2007,May,29,Tue
5,2007-05-29,CULEX RESTUANS,T045,41.921600,-87.666455,2,0,2007,May,29,Tue
6,2007-05-29,CULEX RESTUANS,T046,41.891118,-87.654491,1,0,2007,May,29,Tue
7,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,1,0,2007,May,29,Tue
8,2007-05-29,CULEX RESTUANS,T048,41.867108,-87.654224,2,0,2007,May,29,Tue
9,2007-05-29,CULEX RESTUANS,T049,41.896282,-87.655232,1,0,2007,May,29,Tue


In [4]:
# Create function to return cooodrinates which can be plotted in 2d
def merc(coords):
    coordinates = literal_eval(coords)
    lat = coordinates[0]
    lon = coordinates[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (y, x)

In [5]:
# Create column so function can work properly
train['coords'] = '(' + train['latitude'].astype(str) + ', ' + train['longitude'].astype(str) + ')'

In [6]:
# Create the mercator coordinates
train['coords_lat'] = train['coords'].apply(lambda x: merc(x)[0])
train['coords_long'] = train['coords'].apply(lambda x: merc(x)[1])

In [7]:
# check there are no duplicates
train.duplicated('coords').sum()

10368

In [8]:
# drop 'coords' column as its not needed
train.drop(columns='coords', inplace=True)
train.reset_index(drop=True);

#### There are no duplicated coordinates

In [9]:
train.head()

,date,species,trap,latitude,longitude,nummosquitos,wnvpresent,year,month,day,day_of_week,coords_lat,coords_long
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0,2007,May,29,Tue,5.154195e+06,-9.773962e+06
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,1,0,2007,May,29,Tue,5.154195e+06,-9.773962e+06
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,1,0,2007,May,29,Tue,5.160229e+06,-9.770431e+06
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0,2007,May,29,Tue,5.157099e+06,-9.776613e+06
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,4,0,2007,May,29,Tue,5.157099e+06,-9.776613e+06


In [10]:
# Investigate and see where and when mosquito collection occured
train.loc[((train['species'] == 'CULEX RESTUANS') |
            (train['species'] == 'CULEX PIPIENS')) &
            (train['year'] == 2013)].groupby(['month','day', 'day_of_week']).sum()

latitude    longitude  nummosquitos  wnvpresent  \
month day day_of_week                                                       
Aug   1   Thu          3766.010821 -7892.326735           611           7   
      8   Thu          4183.405029 -8769.543218          1057          15   
      15  Thu          3726.691495 -7806.779365          1001          10   
      22  Thu          3808.512883 -7981.680775           994          16   
      29  Thu          2805.998484 -5877.809688          1027          19   
Jul   8   Mon          3098.601414 -6491.868368          1211           0   
      12  Fri          4101.627801 -8595.541767          2630           5   
      19  Fri          3725.060579 -7805.979698          1957           7   
      25  Thu          3432.406516 -7193.038846          1050           1   
Jun   7   Fri          2008.821017 -4210.322779           236           0   
      14  Fri          2931.131923 -6141.620438          1179           0   
      21  Fri          2804.828171 -5877.507219           923           0   
      27  Thu          1131.638369 -2368.869406           143           0   
      28  Fri          1044.703335 -2192.021805           190           0   
Sep   6   Fri          3348.362113 -7017.242638          1343          18   
      12  Thu          3014.312278 -6315.193689          1113          12   
      19  Thu          2385.885715 -4999.052219           595           4   
      26  Thu          1673.090327 -3507.747903           196           1   

                         year    coords_lat   coords_long  
month day day_of_week                                      
Aug   1   Thu          181170  4.623965e+08 -8.785698e+08  
      8   Thu          201300  5.136166e+08 -9.762211e+08  
      15  Thu          179157  4.576360e+08 -8.690467e+08  
      22  Thu          183183  4.676325e+08 -8.885166e+08  
      29  Thu          134871  3.445890e+08 -6.543148e+08  
Jul   8   Mon          148962  3.805069e+08 -7.226715e+08  
      12  Fri          197274  5.036271e+08 -9.568513e+08  
      19  Fri          179157  4.573924e+08 -8.689577e+08  
      25  Thu          165066  4.214668e+08 -8.007254e+08  
Jun   7   Fri           96624  2.466536e+08 -4.686910e+08  
      14  Fri          140910  3.599425e+08 -6.836821e+08  
      21  Fri          134871  3.444141e+08 -6.542811e+08  
      27  Thu           54351  1.389931e+08 -2.637013e+08  
      28  Fri           50325  1.282327e+08 -2.440148e+08  
Sep   6   Fri          161040  4.111380e+08 -7.811559e+08  
      12  Thu          144936  3.701418e+08 -7.030041e+08  
      19  Thu          114741  2.929624e+08 -5.564919e+08  
      26  Thu           80520  2.054061e+08 -3.904807e+08

In [11]:
# Investigate and see where and when spraying occured
spray.loc[(spray['year'] == 2013)].groupby(['month','day']).count()

date  time  latitude  longitude  year  wday  hour  coords_lat  \
month day                                                                  
7     17   2202  2202      2202       2202  2202  2202  2202        2202   
      25   1607  1607      1607       1607  1607  1607  1607        1607   
8     8    1195  1195      1195       1195  1195  1195  1195        1195   
      15   2668  2668      2668       2668  2668  2668  2668        2668   
      16    141   141       141        141   141   141   141         141   
      22   1587  1587      1587       1587  1587  1587  1587        1587   
      29   2302  2302      2302       2302  2302  2302  2302        2302   
9     5     924   924       924        924   924   924   924         924   

           coords_long  
month day               
7     17          2202  
      25          1607  
8     8           1195  
      15          2668  
      16           141  
      22          1587  
      29          2302  
9     5            924

---

#### Create a list of logical dates when traps were checked and spraying occured

In [12]:
graphs = [(['Jun',7],['No Spraying']),
            (['Jun',14],['No Spraying']),
            (['Jun',21],['No Spraying']),
            (['Jun',27],['No Spraying']),
            (['Jun',28],['No Spraying']),
            (['Jul',8],['No Spraying']),
            (['Jul',12],['No Spraying']),
            (['Jul',12],[7,17]),
            (['Jul',19],[7,17]),
            (['Jul',19],[7,25]),
            (['Jul',25],[7,25]),
            (['Aug',1],['No Spraying']),
            (['Aug',8],['No Spraying']),
            (['Aug',8],[8,8]),
            (['Aug',15],[8,8]),
            (['Aug',15],[8,15,8,16]),
            (['Aug',22],[8,15,8,16]),
            (['Aug',22],[8,22]),
            (['Aug',29],[8,22]),
            (['Aug',29],[8,29,9,5]),
            (['Sep',6],[8,29,9,5]),
            (['Sep',6],['No Spraying']),
            (['Sep',12],['No Spraying']),
            (['Sep',19],['No Spraying']),
            (['Sep',26],['No Spraying'])
        ]

In [13]:
# Function to get train data to plot
def get_train_data(train, mon, day):
    t = pd.DataFrame(train.loc[((train['species'] == 'CULEX RESTUANS') |
            (train['species'] == 'CULEX PIPIENS')) &
            (train['year'] == 2013) & (train['month'] == mon) & 
            (train['day'] == day)].groupby(['day','trap','latitude','longitude']).sum())
    return t

In [14]:
# Function to get spray data to plot
def get_spray_data(spray, mon, day):
    return pd.DataFrame(spray.loc[(spray['year'] == 2013) & 
                                     (spray['day'] == day) &
                                     (spray['month'] == mon)].groupby(['day','latitude','longitude']).sum())

[I used the Code from this blog posting to help map chicago with Bokeh](https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10)

In [15]:
# function to plot spray and collection data onto map of chicago
def create_bokeh(train, spray, graphs):

    count = 1
    for i in graphs:
        # Set map dimensions
#         p = figure(x_range=(-9780700, -9745000), y_range=(5130000, 5160000), x_axis_type="mercator", y_axis_type="mercator", 
#                plot_width=800, plot_height=600)
        p = figure(x_range=(-9770700, -9745000), y_range=(5105000, 5160000), x_axis_type="mercator", y_axis_type="mercator", 
               plot_width=800, plot_height=600)
        p.add_tile(get_provider('CARTODBPOSITRON'))
        
        # Get graph data for correct time period
        train_graph_data = get_train_data(train, i[0][0],i[0][1])

        # plot trap and wnv data on the map
        p.circle(x=train_graph_data['coords_long'],
                 y=train_graph_data['coords_lat'], 
                 # Higher quantities of mosquitos the bigger the red circle
                 size=train_graph_data['nummosquitos'],
                 line_color="#FF0000", 
                 fill_color="#FF0000",
                 fill_alpha=0.05)
        p.circle(x=train_graph_data['coords_long'],
                 y=train_graph_data['coords_lat'], 
                 size=train_graph_data['wnvpresent']*25,
                 line_color="#000000", 
                 fill_color="#000000",
                 fill_alpha=0.5)
        
        # Plot the spray data
        
        # Its not always present so if its missing error is thrown and program continues
        try:
            spray_graph_data1 = get_spray_data(spray,i[1][0],i[1][1])

            p.circle(x=spray_graph_data1['coords_long'],
            y=spray_graph_data1['coords_lat'], 
            size=0.5,
            line_color="#26418F", 
            fill_color="#26418F",
            fill_alpha=0.05)
        except:
            pass
        
        try:
            spray_graph_data2 = get_spray_data(spray,i[1][2],i[1][3])

            p.circle(x=spray_graph_data2['coords_long'],
            y=spray_graph_data2['coords_lat'], 
            size=0.5,
            line_color="#26418F", 
            fill_color="#26418F",
            fill_alpha=0.05)
        except:
            pass
        

#         Save an image to file
#         Commented out because i dont want to overwrite files
#         name ="./assets/bokeh/plot_" + str(count) + "_b.png"
#         export_png(p, filename=name, width=800, height=600)
        count += 1

In [16]:
# Create graphs
create_bokeh(train, spray, graphs)

![Spray / Spread gif](assets/bokeh/spray_and_train_visual_b.gif)

In [17]:
# Images created a Gif to show the transition from June 7th to September 26th

#### Observations

+ There is not much correlation between the trap data and spray data
+ The spraying doesnt seem to react to the presence of WNV.
+ It is widespread though and might be more to do with making sure the area of Chicago is sprayed.
+ The spray areas are the high density areas where people live.